In [1]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
.appName("ShobhitApp") \
.getOrCreate()

In [2]:
from pyspark.sql.functions import col,explode,arrays_zip

In [3]:
%%sh
hdfs dfs -cat /user/itv000197/nestedJson1row.json

[
  {
    "id": "0001",
    "type": "donut",
    "name": "Cake",
    "ppu": 0.55,
    "batters": {
      "batter": [
        {
          "id": "1001",
          "type": "Regular"
        },
        {
          "id": "1002",
          "type": "Chocolate"
        },
        {
          "id": "1003",
          "type": "Blueberry"
        },
        {
          "id": "1004",
          "type": "Devil's Food"
        }
      ]
    },
    "topping": [
      {
        "id": "5001",
        "type": "None"
      },
      {
        "id": "5002",
        "type": "Glazed"
      },
      {
        "id": "5005",
        "type": "Sugar"
      },
      {
        "id": "5007",
        "type": "Powdered Sugar"
      },
      {
        "id": "5006",
        "type": "Chocolate with Sprinkles"
      },
      {
        "id": "5003",
        "type": "Chocolate"
      },
      {
        "id": "5004",
        "type": "Maple"
      }
    ]
  }
]

In [4]:
df = spark.read.option("multiline","true").json("/user/itv000197/nestedJson1row.json")

In [5]:
#df.schema.json()
df.schema.simpleString()

'struct<batters:struct<batter:array<struct<id:string,type:string>>>,id:string,name:string,ppu:double,topping:array<struct<id:string,type:string>>,type:string>'

In [6]:
df.printSchema()

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



In [7]:
df.show()

+--------------------+----+----+----+--------------------+-----+
|             batters|  id|name| ppu|             topping| type|
+--------------------+----+----+----+--------------------+-----+
|[[[1001, Regular]...|0001|Cake|0.55|[[5001, None], [5...|donut|
+--------------------+----+----+----+--------------------+-----+



In [8]:
df_new = df.select("id","name","ppu","type","batters.batter","topping")

In [9]:
df_new.show()

+----+----+----+-----+--------------------+--------------------+
|  id|name| ppu| type|              batter|             topping|
+----+----+----+-----+--------------------+--------------------+
|0001|Cake|0.55|donut|[[1001, Regular],...|[[5001, None], [5...|
+----+----+----+-----+--------------------+--------------------+



In [10]:
df_new1 = df_new.select( "id","name","ppu","type", col("batter"),explode(col("topping")).alias("topping_exploded"))

In [11]:
df_new1.show(truncate=True)

+----+----+----+-----+--------------------+--------------------+
|  id|name| ppu| type|              batter|    topping_exploded|
+----+----+----+-----+--------------------+--------------------+
|0001|Cake|0.55|donut|[[1001, Regular],...|        [5001, None]|
|0001|Cake|0.55|donut|[[1001, Regular],...|      [5002, Glazed]|
|0001|Cake|0.55|donut|[[1001, Regular],...|       [5005, Sugar]|
|0001|Cake|0.55|donut|[[1001, Regular],...|[5007, Powdered S...|
|0001|Cake|0.55|donut|[[1001, Regular],...|[5006, Chocolate ...|
|0001|Cake|0.55|donut|[[1001, Regular],...|   [5003, Chocolate]|
|0001|Cake|0.55|donut|[[1001, Regular],...|       [5004, Maple]|
+----+----+----+-----+--------------------+--------------------+



In [12]:
df_new1.select( "id","name","ppu","type",col("topping_exploded"),explode(col("batter")).alias("batter_exploded"))

id,name,ppu,type,topping_exploded,batter_exploded
0001,Cake,0.55,donut,"[5001, None]","[1001, Regular]"
0001,Cake,0.55,donut,"[5001, None]","[1002, Chocolate]"
0001,Cake,0.55,donut,"[5001, None]","[1003, Blueberry]"
0001,Cake,0.55,donut,"[5001, None]","[1004, Devil's Food]"
0001,Cake,0.55,donut,"[5002, Glazed]","[1001, Regular]"
0001,Cake,0.55,donut,"[5002, Glazed]","[1002, Chocolate]"
0001,Cake,0.55,donut,"[5002, Glazed]","[1003, Blueberry]"
0001,Cake,0.55,donut,"[5002, Glazed]","[1004, Devil's Food]"
0001,Cake,0.55,donut,"[5005, Sugar]","[1001, Regular]"
0001,Cake,0.55,donut,"[5005, Sugar]","[1002, Chocolate]"


#### Explode multiple column as zip

In [13]:
df_new.withColumn("tmp", arrays_zip("batter", "topping"))\
.withColumn("tmp", explode("tmp"))\
.select("id","name","ppu","type",col("tmp.batter"), col("tmp.topping"))

id,name,ppu,type,batter,topping
0001,Cake,0.55,donut,"[1001, Regular]","[5001, None]"
0001,Cake,0.55,donut,"[1002, Chocolate]","[5002, Glazed]"
0001,Cake,0.55,donut,"[1003, Blueberry]","[5005, Sugar]"
0001,Cake,0.55,donut,"[1004, Devil's Food]","[5007, Powdered S..."
0001,Cake,0.55,donut,null,"[5006, Chocolate ..."
0001,Cake,0.55,donut,null,"[5003, Chocolate]"
0001,Cake,0.55,donut,null,"[5004, Maple]"
